In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

# 1. Sesión con los paquetes que ya verificamos
spark = SparkSession.builder \
    .appName("TemperatureAlertsFinal_Snapshot5") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.0") \
    .getOrCreate()

In [2]:
schema = StructType([
    StructField("sensor_id", StringType(), True),
    StructField("temperature", DoubleType(), True),
    StructField("timestamp", StringType(), True)
])

In [3]:
df_kafka_temp = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "kafka:29092") \
    .option("subscribe", "temperature-sensors") \
    .option("startingOffsets", "earliest") \
    .load()

In [ ]:
temp_alerts = df_kafka_temp.select(
    from_json(col("value").cast("string"), schema).alias("data")
).select("data.*") \
 .withColumn("ALERTA_CALOR", col("temperature") > 25.0)

In [ ]:
query_temp = temp_alerts.writeStream \
    .outputMode("append") \
    .format("console") \
    .start()